# 🏷️ **Clasificación de Revisiones de Productos en un Ecommerce con Transformers** 🛒📊

El notebook está enfocado en procesamiento de texto con Transformers para clasificación de sentimientos en reseñas de ecommerce en portugués. Los próximos pasos después de descargar y descomprimir los datasets son:

## 📖 **Índice** 📑
1️⃣ [Introducción](#introduccion)<br>
2️⃣ [Problema del Negocio](#problema)<br>
3️⃣ [Análisis de Datos](#analisis)<br>
4️⃣ [Preprocesamiento de Datos](#preprocesamiento)<br>
5️⃣ [Entrenamiento del Modelo](#entrenamiento)<br>
6️⃣ [Evaluación y Predicción](#evaluacion)

---


## 📝 **1. Introducción**  <a id='introduccion'></a>

En este proyecto, utilizamos un modelo **Transformer** para clasificar revisiones de productos en un ecommerce. Nuestro objetivo es detectar automáticamente si una reseña es **positiva** 😊 o **negativa** 😞 basándonos en el texto ingresado por los usuarios.

⚠️ **Nota**: Este proyecto **NO** se enfoca en un análisis exploratorio detallado de los datos, sino en el desarrollo y ajuste fino del modelo de clasificación.

---


## ❓ **2. Problema del Negocio**  <a id='problema'></a>

💡 **Objetivo**: Clasificar automáticamente las revisiones de productos en el ecommerce **Olist** según el sentimiento expresado en el texto.

🔍 **Beneficio**: Permite a la empresa entender mejor la satisfacción de los clientes y tomar acciones basadas en los comentarios recibidos.

---

## 🖥️ **3. Configuración del Entorno**  <a id='configuracion'></a>

Para ejecutar este proyecto, es necesario instalar las siguientes librerías:

In [ ]:
# Instalación bibliotecas necesarias

# Transformers para trabajar con modelos de NLP
%pip install transformers datasets

# Acceso a datasets desde Kaggle
%pip install kaggle

# Aceleración del entrenamiento en GPU/TPU
%pip install accelerate -U

# Librerías principales para modelos de deep learning
%pip install torch
%pip install transformers[torch]

# Interfaz gráfica para pruebas de modelo
%pip install gradio

# Instalación de bibliotecas esenciales para análisis de datos y aprendizaje automático
%pip install  install pandas t
%pip install tensorflow 
%pip install scikit-learn

# Mejorar la experiencia en Jupyter Notebook
%pip install --upgrade jupyter ipywidgets

# Instalar tf-keras para compatibilidad con TensorFlow
%pip install tf-keras

🔹 **Desactivar los warnings**

In [2]:
import warnings
import tensorflow as tf
from transformers.utils import logging

# Desactivar warnings de TensorFlow
tf.get_logger().setLevel('ERROR')

# Desactivar warnings de Transformers
logging.set_verbosity_error()

# Desactivar warnings generales de Python
warnings.filterwarnings("ignore")

🔹 **Configuración de la API de Kaggle** (si los datos provienen de Kaggle)

Seleccionar el archivo kaggle.json y moverlo a la carpeta .kaggle de tu usuario.

In [3]:
import os
import shutil

# 📌 Pedir manualmente la ruta de kaggle.json
file_path = input("🔍 Ingresa la ruta completa de tu archivo kaggle.json: ")

# 📌 Validar si la ruta ingresada es correcta
if not os.path.exists(file_path):
    print("⚠️ ERROR: La ruta ingresada no es válida. Verifica el archivo y vuelve a intentarlo.")
else:
    # 📌 Ruta destino en .kaggle
    kaggle_dir = os.path.join(os.path.expanduser("~"), ".kaggle")
    os.makedirs(kaggle_dir, exist_ok=True)

    destino = os.path.join(kaggle_dir, "kaggle.json")
    shutil.move(file_path, destino)
    print(f"✅ kaggle.json movido a: {destino}")


✅ kaggle.json movido a: C:\Users\esaa2\.kaggle\kaggle.json


O simplemente Crea la carpeta .kaggle en "tu usuario" de Windows y copia manualmente kaggle.json a la carpeta .kaggle.

🔹 Descargar el dataset desde Kaggle

In [4]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Configurar Kaggle API
api = KaggleApi()
api.authenticate()

# Descargar dataset
dataset_name = "olistbr/brazilian-ecommerce"
save_path = "datasets"

if not os.path.exists(save_path):
    os.makedirs(save_path)

api.dataset_download_files(dataset_name, path=save_path, unzip=True)
print("Dataset descargado y extraído en:", save_path)


Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
Dataset descargado y extraído en: datasets


🔹 Verificar que todo funciona

In [5]:
import os

kaggle_dir = os.path.join(os.path.expanduser("~"), ".kaggle")
json_path = os.path.join(kaggle_dir, "kaggle.json")

if os.path.exists(json_path):
    print(f"✅ kaggle.json está en: {json_path}")
else:
    print("⚠️ ERROR: No se encontró kaggle.json en la ubicación esperada.")


✅ kaggle.json está en: C:\Users\esaa2\.kaggle\kaggle.json


## 📊 **4. Análisis de Datos**  <a id='analisis'></a>

🔹 **Carga de Datos**: Importamos el dataset con reseñas de productos "olist_order_reviews_dataset.csv"

In [6]:
import pandas as pd
import os

# Definir la ruta correcta del dataset descargado
dataset_path = "datasets/olist_order_reviews_dataset.csv"

# Verificar si el archivo existe
if os.path.exists(dataset_path):
    df = pd.read_csv(dataset_path)
    print("✅ Dataset cargado correctamente.")
else:
    print("⚠️ ERROR: No se encontró el archivo. Verifica la ruta.")

df.head()


✅ Dataset cargado correctamente.


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


🔹 Verificamos la calidad de los datos.

In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB
None


## 🛠️ **5. Preprocesamiento de Datos**  <a id='preprocesamiento'></a>

⚠️ **Nota**: No realizamos un análisis profundo de datos, sino una **limpieza justa y necesaria** para preparar los datos para el modelo.


🔹 Filtrar datos y crear la columna de sentimiento<br>
🔹 **Limpieza**: Eliminamos filas con valores nulos.<br> 
🔹 **Creación de Etiquetas**: Convertimos las calificaciones en **positivas (1)** o **negativas (0)**.

In [8]:
# Eliminar filas con comentarios vacíos y Selecciona solamente las líneas con comentarios y puntuaciones
df = df.dropna(subset=['review_comment_message', 'review_score'])

# Crear columna de sentimientos (1 = Positivo, 0 = Negativo)
df['sentiment'] = df['review_score'].apply(lambda score: 1 if score >= 4 else 0)

df[['review_comment_message', 'review_score', 'sentiment']].head()


,review_comment_message,review_score,sentiment
3,Recebi bem antes do prazo estipulado.,5,1
4,Parabéns lojas lannister adorei comprar pela I...,5,1
9,aparelho eficiente. no site a marca do aparelh...,4,1
12,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",4,1
15,"Vendedor confiável, produto ok e entrega antes...",5,1


## 🤖 **6. Entrenamiento del Modelo**  <a id='entrenamiento'></a>

🔹 **División de Datos**: Separación en conjuntos de entrenamiento y prueba.

In [9]:
from sklearn.model_selection import train_test_split

# Reducir el tamaño de los datos para acelerar el entrenamiento
df_sample = df.sample(frac=0.1, random_state=42)

# Dividir en conjuntos de entrenamiento (90%) y prueba (10%)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_sample['review_comment_message'], df_sample['sentiment'], test_size=0.1, random_state=42
)

print(f"Datos de entrenamiento: {len(train_texts)}")
print(f"Datos de prueba: {len(test_texts)}")


Datos de entrenamiento: 3688
Datos de prueba: 410


🔹 **Tokenización**: Preparación del texto para el modelo Transformer (Tokenización con AutoTokenizer).

In [10]:
from transformers import AutoTokenizer

# Usar un tokenizer preentrenado en portugués
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Tokenizar los textos de entrenamiento y prueba
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

print("✅ Tokenización completada.")



✅ Tokenización completada.


🔹 Creación y compilación del modelo

In [11]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

# Cargar el modelo de transformers con 2 clases (positivo/negativo)
model = TFAutoModelForSequenceClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased", num_labels=2
)

# Configurar optimizador y función de pérdida
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compilar el modelo
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
print("✅ Modelo compilado correctamente.")

✅ Modelo compilado correctamente.


🔹 Convertir datos en `tf.data.Dataset` para entrenamiento

Crear datasets en formato TensorFlow

In [12]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((
        dict(train_encodings), train_labels
    ))
    .shuffle(1000)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset.from_tensor_slices((
        dict(test_encodings), test_labels
    ))
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

print("✅ Datos convertidos a `tf.data.Dataset`.")

✅ Datos convertidos a `tf.data.Dataset`.


🔹 Entrenamiento del modelo

In [13]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=1 # 3 (Reducir la cantidad de epochs )
)

print("✅ Entrenamiento completado.")

116/116 [==============================] - 1482s 12s/step - loss: 0.3269 - accuracy: 0.8682 - val_loss: 0.2730 - val_accuracy: 0.9073
✅ Entrenamiento completado.


🔹 Guardado del modelo

In [14]:
model_dir = "sentiment_analysis_model"

# Guardar el modelo y el tokenizer
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

print(f"✅ Modelo guardado en: {model_dir}")

✅ Modelo guardado en: sentiment_analysis_model


🔹 Cargar el modelo guardado para predicciones

In [15]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# Cargar modelo y tokenizer
model_dir = "sentiment_analysis_model"
tokenizer = BertTokenizer.from_pretrained(model_dir)
model = TFBertForSequenceClassification.from_pretrained(model_dir)

# Recompilar el modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

print("✅ Modelo cargado y recompilado correctamente.")

✅ Modelo cargado y recompilado correctamente.


## 🎯 **7. Evaluación y Predicción**  <a id='evaluacion'></a>

🔹 Evaluación del modelo en los datos de prueba

In [16]:
# Evaluar en datos de prueba
loss, accuracy = model.evaluate(test_dataset)
print(f"📊 Precisión del modelo en test: {accuracy:.4f}")

13/13 [==============================] - 39s 2s/step - loss: 0.2730 - accuracy: 0.9073
📊 Precisión del modelo en test: 0.9073


🔹 Predicción con nuevos comentarios

In [17]:
import tensorflow as tf

def predict_sentiment(text):
    # Tokenizar el texto
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=128)
    
    # Hacer predicción
    logits = model(**inputs).logits
    predicted_class = tf.argmax(logits, axis=1).numpy()[0]

    # Convertir la predicción en etiqueta
    sentiment_label = "Positivo 😊" if predicted_class == 1 else "Negativo 😞"
    return sentiment_label

# Prueba con ejemplos
ejemplo1 = "O produto é maravilhoso, chegou rápido e tem ótima qualidade!"
ejemplo2 = "Não gostei, o produto veio com defeito e o atendimento foi péssimo."

print(f"🔎 Predicción 1: {predict_sentiment(ejemplo1)}")
print(f"🔎 Predicción 2: {predict_sentiment(ejemplo2)}")

🔎 Predicción 1: Positivo 😊
🔎 Predicción 2: Negativo 😞


## 🎯 **8. Evaluación y Predicción con GRADIO**  <a id='gradio'></a>

In [18]:
# 💡 Importamos Gradio para crear la interfaz gráfica interactiva
import gradio as gr
import tensorflow as tf

# 📌 Definimos la función que Gradio usará para hacer predicciones
def classify_review(text):
    """
    Función que clasifica una reseña de cliente como 'Positiva' o 'Negativa'
    utilizando el modelo BERT entrenado.
    """
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True)
    outputs = model(inputs)[0]
    prediction = tf.nn.softmax(outputs, axis=1)
    label = "Positiva" if tf.argmax(prediction, axis=1).numpy()[0] == 1 else "Negativa"
    return label

# 🎛️ Creamos la interfaz de usuario con Gradio
iface = gr.Interface(
    fn=classify_review,  # Función de clasificación
    inputs="text",  # Entrada: Texto del usuario
    outputs="label",  # Salida: Predicción de sentimiento
    title="🔍 Clasificación de Reseñas con BERT",
    description="Ingrese una reseña y el modelo la clasificará como Positiva o Negativa.",
    theme="default"
)

# 🚀 Lanzamos la interfaz
iface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## 🏆 **Conclusión**

✔️ Se ha entrenado un modelo Transformer para clasificar reseñas de productos.<br>
✔️ El modelo predice si una reseña es positiva o negativa con precisión.<br>
✔️ Puede ser implementado en un ecommerce para análisis automatizado de satisfacción del cliente.

**🚀 Próximos Pasos:**

🔹 Mejorar el modelo con más datos y entrenamiento prolongado.<br>
🔹 Aplicar técnicas de reducción de ruido en los textos.<br>
🔹 Evaluar el modelo con métricas adicionales como `F1-score` y `AUC-ROC`.<br>
🔹 **Para un ajuste fino más robusto:**<br>
   1️⃣ Aumentar el número de epochs (ejemplo: `epochs=3 o 5`).<br>
   2️⃣ Usar congelación de capas para entrenar solo algunas partes del modelo BERT.<br>
   3️⃣ Aplicar estrategias de regularización como `dropout` para evitar sobreajuste.<br><br>

🔎 **Conclusión**: El ajuste fino es el proceso de entrenar un modelo preentrenado en un nuevo conjunto de datos con una tarea específica. En este caso, hemos afinado BERT para clasificar reseñas en Olist. 🚀

---

📢 **Gracias por revisar este proyecto!** 🙌